### Runs an extended Kalman filter using IHME SEIIR predictions along with measurement data of confirmed Covid-19 case counts (from New York Times data) and Facebook symptom data (loss of smell/taste, from Covid-19 Symptom Challenge) to generate updated 7-day predictions of case counts for counties in New York State.

Developed by the University of Washington team of Les Atlas, Abraham Flaxman and Michael Rhoads.

S - Susceptible
E - Exposed
I1 - Presymptomatic
I2 - Symptomatic
R - Recovered

In [1]:
#%load_ext autoreload
#%autoreload

In [2]:
#%reset

In [3]:
import math
import datetime

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import data_sets
import seiir_compartmental

In [4]:
# functions to support the Kalman filtering
def get_predicts_prior(day, seiir):
    x_hat = np.array([[seiir['S'].loc[day]],
                      [seiir['E'].loc[day]],
                      [seiir['I1'].loc[day]],
                      [seiir['I2'].loc[day]],
                      [seiir['R'].loc[day]]])

    beta_k = seiir['beta_pred'].loc[day]

    return x_hat, beta_k


def step_seiir(x_hat, constants, beta_k, days=7):
    s_dict = {'S': x_hat[0, 0],
              'E': x_hat[1, 0],
              'I1': x_hat[2, 0],
              'I2': x_hat[3, 0],
              'R': x_hat[4, 0]}

    s = pd.Series(s_dict)

    for i in range(days):
        infectious = s.loc['I1'] + s.loc['I2']
        s = seiir_compartmental.compartmental_covid_step(s, s.sum(),
                                                         infectious,
                                                         constants['alpha'],
                                                         beta_k,
                                                         constants['gamma1'],
                                                         constants['gamma2'],
                                                         constants['sigma'],
                                                         constants['theta'])
    x_hat_future_prior = np.array([[s.loc['S']],
                                   [s.loc['E']],
                                   [s.loc['I1']],
                                   [s.loc['I2']],
                                   [s.loc['R']]])

    return x_hat_future_prior


def predict_step(x_hat_k1_prior, P, Q, beta_k, constants):
    S = x_hat_k1_prior[0, 0]
    E = x_hat_k1_prior[1, 0]
    I1 = x_hat_k1_prior[2, 0]
    I2 = x_hat_k1_prior[3, 0]
    R = x_hat_k1_prior[4, 0]
    N = S + E + I1 + I2 + R
    alpha = constants['alpha']
    sigma = constants['sigma']
    gamma1 = constants['gamma1']
    gamma2 = constants['gamma2']

    part_f_S = np.array([[-beta_k * math.pow(I1 + I2, alpha) / N],
                         [beta_k * math.pow(I1 + I2, alpha) / N],
                         [0],
                         [0],
                         [0]])

    part_f_E = np.array([[0],
                         [-sigma],
                         [sigma],
                         [0],
                         [0]])

    part_f_I1 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [-gamma1],
                          [gamma1],
                          [0]])

    part_f_I2 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [0],
                          [-gamma2],
                          [gamma2]])

    part_f_R = np.array([[0],
                         [0],
                         [0],
                         [0],
                         [0]])

    # 5x5
    f_jacob = np.concatenate([part_f_S, part_f_E, part_f_I1, part_f_I2,
                              part_f_R], axis=1)

    # 5x5
    # P_k1_prior = f_jacob * P * f_jacob^T + Q
    P_k1_prior = np.matmul(np.matmul(f_jacob, P), np.transpose(f_jacob)) + Q
    return P_k1_prior


def update_step(x_hat, x_hat_k1, P_k1, Rn, rho1, rho2, z_k):
    # 5x5
    ep = 10**-8
    
    H = np.array([[ep, 0, 0, 0, 0],
                  [0, ep, 0, 0, 0],
                  [0, 0, ep, rho1, 0],
                  [0, 0, 0, rho2, 0],
                  [0, 0, 0, 0, ep]])
    
    
    # Si = H * P_k1 * H^T + Rn
    Si = np.matmul(np.matmul(H, P_k1), np.transpose(H)) + Rn

    # K_new = P_k1 * H^T * Si^(-1)
    K_new = np.matmul(np.matmul(P_k1, np.transpose(H)), np.linalg.inv(Si))
    y_new = np.matmul(H, x_hat)

    # 5x1
    diff = z_k - y_new

    x_hat_k1_post = x_hat_k1 + np.matmul(K_new, diff)

    P_k1_post = P_k1 - np.matmul(np.matmul(K_new, Si), np.transpose(K_new))


    return x_hat_k1_post, P_k1_post


def get_data_sets(state, fips=None):
    # the post hoc seiir model predictions provided by Prof Flaxman without
    # any kalman filtering:
    seiir = pd.read_csv(r'data/seiir_compartments_post-hoc_ny_state.csv', header=0,
                        index_col='date', parse_dates=True)
    

    fb_data = data_sets.create_symptom_df()
    fb_data_val = data_sets.create_symptom_df(valid=True)

    if fips is None:
        case_data = data_sets.create_case_df_state()
        case_data_geo = case_data.loc[state]['case_rate'].copy()
        fb_data_geo = fb_data.loc[state].groupby('date').sum().copy()
        fb_data_val_geo = fb_data_val.loc[state].groupby('date').sum().copy()

    else:
        case_data = data_sets.create_case_df_county()
        case_data_geo = case_data.loc[fips]['case_rate'].copy()
        
        if fips == 'New York City':
            nyc_fips = ['36005', '36061', '36047', '36085']
            fb_data_geo = fb_data.loc[(slice(None), '36081'), :].copy()
            fb_data_geo = fb_data_geo.mean(level='date')
            fb_data_val_geo = fb_data_val.loc[(slice(None), '36081'), :].copy()
            fb_data_val_geo = fb_data_val.mean(level='date')
            for borough in nyc_fips:
                fb_data_geo += fb_data.loc[(slice(None), borough), :].copy().mean(level='date')
                fb_data_val_geo += fb_data_val.loc[(slice(None), borough), :].copy().mean(level='date')
        else:
            fb_data_geo = fb_data.loc[(slice(None), fips), :].copy()
            fb_data_val_geo = fb_data_val.loc[(slice(None), fips), :].copy()
            # collapse down to a single index column (date)
            fb_data_geo = fb_data_geo.mean(level='date')
            fb_data_val_geo = fb_data_val_geo.mean(level='date')


    return seiir, fb_data_geo, fb_data_val_geo, case_data_geo


def calc_fb_ma7(fb_data):
    """
    Returns a Pandas series
    """
    # the fb_data is a DataFrame while the case_data is a Series
    fb_ma7 = fb_data.rolling(window=7).mean()
    fb_ma7 = fb_ma7.iloc[6:, :]
    prop_ma7 = fb_ma7['num_stl'].div(fb_ma7['n'])

    return prop_ma7, fb_ma7['n'].copy(), fb_ma7['num_stl'].copy()

In [5]:
# set constants
K0 = datetime.date(2020, 4, 12)

the_state = 'NY'
the_county = data_sets.get_fips(the_state, 'Nassau')
#the_county = 'New York City'
constants = {
    'alpha': 0.948786,
    'gamma1': 0.500000,
    'gamma2': 0.662215,
    'sigma': 0.266635,
    'theta': 6.000000
    }

# set initial values for Kalman filter parameters
P_mult = 1
Q_mult = 1

# Rn is the R noise covariance matrix; it remains constant thru the stepping of the
# Kalman filter
Rn_mult = 5*10**-8

Rn_22 = 10000
Rn_32 = 1000

Rn_23 = 1000
Rn_33 = 100

Rn = Rn_mult * np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, Rn_22, Rn_23, 0],
                         [0, 0, Rn_32, Rn_33, 0],
                         [0, 0, 0, 0, 0]])

Q = Q_mult * np.eye(5)
P = P_mult * np.eye(5)

# generate data
seiir, fb_data, fb_data_val, case_data = get_data_sets(
    data_sets.STATES[the_state], fips=the_county)


if the_county == 'New York City':
    county_pop = 0
    for each in ['36081', '36005', '36061', '36047', '36085']:
        this_count, state_pop = data_sets.get_pops(each)
        county_pop += this_count
else:
    county_pop, state_pop = data_sets.get_pops(the_county)

b = county_pop / state_pop

# calculate moving averages on the fb and case data
case_ma7 = case_data.rolling(window=7).mean()
case_ma7_all = case_ma7.iloc[6:]
prop_ma7, n_ma7, num_stl_ma7 = calc_fb_ma7(fb_data)

prop_ma7_valid, n_ma7_valid, num_stl_ma7_valid = calc_fb_ma7(fb_data_val)

# get starting compartment values for the state level
x_hat_state_k0, beta_k0 = get_predicts_prior(K0, seiir)

# convert to the county level
x_hat_k0 = b * x_hat_state_k0

I2_county = x_hat_k0[3, 0]


rho1 = 0.0001
rho2 = case_ma7_all.loc['2020-04-12'] / I2_county


# create empty dictionaries to hold the estimated values
prop_est = {}
case_est = {}
seiir_pred = {}



# Original data run ----------------
start = K0
d = start
#while d <= datetime.date(2020, 9, 30):
while d <= datetime.date(2020, 10, 23):    

# each cycle of the while loop executes a step

    # get state level compartments
    x_hat_state_k, beta_k = get_predicts_prior(d, seiir)

    # step the state level compartments 7 days forward
    x_hat_state_k1 = step_seiir(x_hat_state_k, constants, beta_k)
    
    # convert the state level compartments to county level values
    x_hat_k = b * x_hat_state_k
    x_hat_k1 = b * x_hat_state_k1


    # get measurements for current day
    
    z_k = np.array([[0],
                    [0],
                    [prop_ma7.loc[d]],
                    [case_ma7_all.loc[d]],
                    [0]])

    # predict step using the stepped fwd SEIIR compartment values 
    P = predict_step(x_hat_k1, P, Q, beta_k, constants)
    print('step -----------')
    print('P:')
    print(P)

    # update step
    x_hat_post, P_post = update_step(x_hat_k, x_hat_k1, P, Rn,
                                     rho1, rho2, z_k)
    print('P_post:')
    print(P_post)

    # store estimated values for proportion and case rate
    indexDate = d + datetime.timedelta(days=7)
    prop_est[indexDate] = rho1 * x_hat_post[3, 0]
    case_est[indexDate] = rho2 * x_hat_post[3, 0]
    seiir_pred[indexDate] = x_hat_k1[3, 0]

    # update the P and d
    P = P_post
    d += datetime.timedelta(days=1)
    
# create pandas series of the estimated case rate
predicted_case = pd.Series(case_est)
predicted_seiir_prior = pd.Series(seiir_pred)

step -----------
P:
[[ 1.13099643 -0.13099643  0.12795814  0.04151346 -0.16947159]
 [-0.13099643  1.20209065 -0.19905236 -0.04151346  0.16947159]
 [ 0.12795814 -0.19905236  1.32109422 -0.25        0.        ]
 [ 0.04151346 -0.04151346 -0.25        1.68852871 -0.43852871]
 [-0.16947159  0.16947159  0.         -0.43852871  1.43852871]]
P_post:
[[ 0.00000000e+00  8.32667268e-17 -1.13797860e-15  6.79317713e-15
   2.52575738e-15]
 [ 5.55111512e-17  0.00000000e+00  1.38777878e-16 -5.75928194e-16
   2.77555756e-17]
 [-1.19348975e-15  1.11022302e-16  1.23736742e+00  7.76799919e-09
   1.38272901e-16]
 [ 6.79317713e-15 -5.82867088e-16  7.76799919e-09  2.22044605e-16
   8.32667268e-16]
 [ 2.49800181e-15  2.77555756e-17  1.26821818e-16  9.43689571e-16
  -4.44089210e-16]]
step -----------
P:
[[ 1.08003451e+00 -8.00345149e-02  1.57346831e-01 -1.57346830e-01
  -1.30827042e-09]
 [-8.00345149e-02  1.08003451e+00 -1.57346831e-01  1.57346830e-01
   1.30827052e-09]
 [ 1.57346831e-01 -1.57346831e-01  1.309

step -----------
P:
[[ 1.07398492e+00 -7.39849174e-02  1.49496548e-01 -1.49496546e-01
  -1.24299855e-09]
 [-7.39849174e-02  1.07398492e+00 -1.49496548e-01  1.49496546e-01
   1.24299847e-09]
 [ 1.49496548e-01 -1.49496548e-01  1.30207803e+00 -3.02078023e-01
  -2.51164689e-09]
 [-1.49496546e-01  1.49496546e-01 -3.02078023e-01  1.30207802e+00
   2.51164697e-09]
 [-1.24299855e-09  1.24299847e-09 -2.51164689e-09  2.51164697e-09
   1.00000000e+00]]
P_post:
[[ 0.00000000e+00 -1.66533454e-16  5.55111512e-17 -2.77555756e-17
  -1.86115638e-24]
 [-1.80411242e-16  4.44089210e-16 -3.60822483e-16  1.30451205e-15
   4.54949337e-24]
 [ 0.00000000e+00 -3.05311332e-16  1.20832279e+00  7.58566082e-09
  -8.27180613e-25]
 [ 0.00000000e+00  1.33226763e-15  7.58566071e-09  0.00000000e+00
   0.00000000e+00]
 [-2.06795153e-24  4.13590306e-24 -6.81886338e-21  0.00000000e+00
   1.11022302e-16]]
step -----------
P:
[[ 1.07379488e+00 -7.37948837e-02  1.49305090e-01 -1.49305089e-01
  -1.24140667e-09]
 [-7.37948837e-

step -----------
P:
[[ 1.07448336e+00 -7.44833590e-02  1.49988400e-01 -1.49988399e-01
  -1.24708812e-09]
 [-7.44833590e-02  1.07448336e+00 -1.49988400e-01  1.49988399e-01
   1.24708821e-09]
 [ 1.49988400e-01 -1.49988400e-01  1.30203418e+00 -3.02034177e-01
  -2.51128255e-09]
 [-1.49988399e-01  1.49988399e-01 -3.02034177e-01  1.30203417e+00
   2.51128245e-09]
 [-1.24708812e-09  1.24708822e-09 -2.51128257e-09  2.51128247e-09
   1.00000000e+00]]
P_post:
[[ 0.00000000e+00 -2.91433544e-16 -1.38777878e-16  9.99200722e-16
  -2.27474668e-24]
 [-3.19189120e-16 -6.66133815e-16  2.77555756e-17 -5.55111512e-17
  -2.06795153e-25]
 [-1.11022302e-16  2.77555756e-17  1.20815575e+00  7.58461233e-09
  -4.13590306e-25]
 [ 1.02695630e-15 -5.55111512e-17  7.58461233e-09 -2.22044605e-16
  -1.65436123e-24]
 [-2.27474668e-24 -2.06795153e-25 -1.70614315e-17 -2.06795153e-24
   1.11022302e-16]]
step -----------
P:
[[ 1.07447508e+00 -7.44750804e-02  1.49981246e-01 -1.49981245e-01
  -1.24702860e-09]
 [-7.44750804e-

step -----------
P:
[[ 1.07603836e+00 -7.60383575e-02  1.51527472e-01 -1.51527471e-01
  -1.25988480e-09]
 [-7.60383575e-02  1.07603836e+00 -1.51527472e-01  1.51527471e-01
   1.25988480e-09]
 [ 1.51527472e-01 -1.51527472e-01  1.30196043e+00 -3.01960423e-01
  -2.51066918e-09]
 [-1.51527471e-01  1.51527471e-01 -3.01960423e-01  1.30196042e+00
   2.51066918e-09]
 [-1.25988480e-09  1.25988480e-09 -2.51066918e-09  2.51066918e-09
   1.00000000e+00]]
P_post:
[[ 2.22044605e-16 -4.16333634e-17  5.55111512e-17 -1.66533454e-16
   0.00000000e+00]
 [-5.55111512e-17  0.00000000e+00  2.77555756e-17 -2.77555756e-17
   0.00000000e+00]
 [ 5.55111512e-17  0.00000000e+00  1.20767036e+00  7.58156504e-09
  -8.27180613e-25]
 [-1.66533454e-16 -2.77555756e-17  7.58156504e-09  0.00000000e+00
   0.00000000e+00]
 [ 2.06795153e-25 -2.06795153e-25 -4.13590306e-25  0.00000000e+00
   1.11022302e-16]]
step -----------
P:
[[ 1.07681075e+00 -7.68107489e-02  1.52284327e-01 -1.52284326e-01
  -1.26617773e-09]
 [-7.68107489e-

step -----------
P:
[[ 1.07857962e+00 -7.85796153e-02  1.53967927e-01 -1.53967926e-01
  -1.28017613e-09]
 [-7.85796153e-02  1.07857962e+00 -1.53967927e-01  1.53967926e-01
   1.28017611e-09]
 [ 1.53967927e-01 -1.53967927e-01  1.30168286e+00 -3.01682853e-01
  -2.50836129e-09]
 [-1.53967926e-01  1.53967926e-01 -3.01682853e-01  1.30168285e+00
   2.50836131e-09]
 [-1.28017613e-09  1.28017611e-09 -2.50836129e-09  2.50836131e-09
   1.00000000e+00]]
P_post:
[[-4.44089210e-16 -1.66533454e-16  5.55111512e-17 -3.60822483e-16
  -6.20385459e-25]
 [-1.80411242e-16  4.44089210e-16 -4.99600361e-16  2.02615702e-15
   1.44756607e-24]
 [ 5.55111512e-17 -5.27355937e-16  1.20679610e+00  7.57607627e-09
  -1.24077092e-24]
 [-3.33066907e-16  1.99840144e-15  7.57607621e-09  0.00000000e+00
   2.06795153e-24]
 [-4.13590306e-25  1.44756607e-24 -3.76491256e-21  2.48154184e-24
   1.11022302e-16]]
step -----------
P:
[[ 1.07836839e+00 -7.83683928e-02  1.53764975e-01 -1.53764974e-01
  -1.27848863e-09]
 [-7.83683928e-

...

step -----------
P:
[[ 1.11279864e+00 -1.12798643e-01  1.83639859e-01 -1.83639857e-01
  -1.52688530e-09]
 [-1.12798643e-01  1.11279864e+00 -1.83639859e-01  1.83639857e-01
   1.52688530e-09]
 [ 1.83639859e-01 -1.83639859e-01  1.29897166e+00 -2.98971659e-01
  -2.48581894e-09]
 [-1.83639857e-01  1.83639857e-01 -2.98971659e-01  1.29897166e+00
   2.48581893e-09]
 [-1.52688530e-09  1.52688529e-09 -2.48581893e-09  2.48581893e-09
   1.00000000e+00]]
P_post:
[[ 0.00000000e+00  1.11022302e-16 -5.55111512e-17  5.55111512e-17
  -1.44756607e-24]
 [ 1.11022302e-16  0.00000000e+00  1.66533454e-16 -5.27355937e-16
   2.06795153e-25]
 [-5.55111512e-17  1.38777878e-16  1.19610242e+00  7.50894336e-09
  -1.24077092e-24]
 [ 5.55111512e-17 -5.27355937e-16  7.50894336e-09  0.00000000e+00
   2.06795153e-24]
 [-1.44756607e-24  0.00000000e+00  8.62044745e-18  2.48154184e-24
   1.11022302e-16]]
step -----------
P:
[[ 1.11243832e+00 -1.12438322e-01  1.83362857e-01 -1.83362856e-01
  -1.52458216e-09]
 [-1.12438322e-

step -----------
P:
[[ 1.11332202e+00 -1.13322020e-01  1.84084330e-01 -1.84084329e-01
  -1.53058085e-09]
 [-1.13322020e-01  1.11332202e+00 -1.84084330e-01  1.84084329e-01
   1.53058095e-09]
 [ 1.84084330e-01 -1.84084330e-01  1.29903315e+00 -2.99033147e-01
  -2.48633029e-09]
 [-1.84084329e-01  1.84084329e-01 -2.99033147e-01  1.29903314e+00
   2.48633029e-09]
 [-1.53058085e-09  1.53058095e-09 -2.48633029e-09  2.48633029e-09
   1.00000000e+00]]
P_post:
[[ 1.11022302e-15 -4.02455846e-16  3.88578059e-16 -9.43689571e-16
  -1.86115638e-24]
 [-3.88578059e-16  0.00000000e+00 -2.77555756e-17  8.32667268e-17
   0.00000000e+00]
 [ 3.60822483e-16 -5.55111512e-17  1.19600027e+00  7.50830192e-09
   4.13590306e-25]
 [-8.88178420e-16  1.11022302e-16  7.50830192e-09  0.00000000e+00
  -2.06795153e-24]
 [-1.65436123e-24 -2.06795153e-25  4.13590306e-25 -1.65436123e-24
   0.00000000e+00]]
step -----------
P:
[[ 1.11410078e+00 -1.14100775e-01  1.84705548e-01 -1.84705547e-01
  -1.53574602e-09]
 [-1.14100775e-

step -----------
P:
[[ 1.11962136e+00 -1.19621355e-01  1.89006353e-01 -1.89006351e-01
  -1.57150530e-09]
 [-1.19621355e-01  1.11962136e+00 -1.89006353e-01  1.89006351e-01
   1.57150529e-09]
 [ 1.89006353e-01 -1.89006353e-01  1.29863732e+00 -2.98637321e-01
  -2.48303903e-09]
 [-1.89006351e-01  1.89006351e-01 -2.98637321e-01  1.29863732e+00
   2.48303914e-09]
 [-1.57150527e-09  1.57150526e-09 -2.48303899e-09  2.48303910e-09
   1.00000000e+00]]
P_post:
[[ 2.22044605e-16 -1.38777878e-17 -1.38777878e-16  4.44089210e-16
  -2.06795153e-25]
 [ 0.00000000e+00  4.44089210e-16  2.22044605e-16 -1.08246745e-15
  -2.68833699e-24]
 [-1.11022302e-16  1.66533454e-16  1.19418766e+00  7.49692292e-09
   8.27180613e-25]
 [ 4.16333634e-16 -1.08246745e-15  7.49692292e-09  0.00000000e+00
  -8.27180613e-25]
 [ 0.00000000e+00 -2.06795153e-24  2.82021713e-17 -4.13590306e-25
   0.00000000e+00]]
step -----------
P:
[[ 1.11973729e+00 -1.19737290e-01  1.89069295e-01 -1.89069294e-01
  -1.57202873e-09]
 [-1.19737290e-

P_post:
[[-2.22044605e-16  5.55111512e-17  5.55111512e-17 -1.38777878e-16
   4.13590306e-25]
 [ 5.55111512e-17 -2.22044605e-16  2.77555756e-17  2.77555756e-17
   0.00000000e+00]
 [ 0.00000000e+00  2.77555756e-17  1.19442270e+00  7.49839829e-09
   0.00000000e+00]
 [-1.38777878e-16  0.00000000e+00  7.49839829e-09  0.00000000e+00
  -8.27180613e-25]
 [ 2.06795153e-25 -4.13590306e-25 -4.30027047e-18 -4.13590306e-25
   1.11022302e-16]]
step -----------
P:
[[ 1.11794260e+00 -1.17942604e-01  1.87665476e-01 -1.87665474e-01
  -1.56035655e-09]
 [-1.17942604e-01  1.11794260e+00 -1.87665476e-01  1.87665474e-01
   1.56035654e-09]
 [ 1.87665476e-01 -1.87665476e-01  1.29860568e+00 -2.98605673e-01
  -2.48277591e-09]
 [-1.87665474e-01  1.87665474e-01 -2.98605673e-01  1.29860567e+00
   2.48277591e-09]
 [-1.56035655e-09  1.56035655e-09 -2.48277591e-09  2.48277591e-09
   1.00000000e+00]]
P_post:
[[ 0.00000000e+00  8.32667268e-17 -1.11022302e-16  3.33066907e-16
   1.24077092e-24]
 [ 1.11022302e-16  0.000000

step -----------
P:
[[ 1.12191715e+00 -1.21917148e-01  1.90779150e-01 -1.90779149e-01
  -1.58624536e-09]
 [-1.21917148e-01  1.12191715e+00 -1.90779150e-01  1.90779149e-01
   1.58624537e-09]
 [ 1.90779150e-01 -1.90779150e-01  1.29853622e+00 -2.98536215e-01
  -2.48219844e-09]
 [-1.90779149e-01  1.90779149e-01 -2.98536215e-01  1.29853621e+00
   2.48219852e-09]
 [-1.58624536e-09  1.58624537e-09 -2.48219844e-09  2.48219852e-09
   1.00000000e+00]]
P_post:
[[ 2.22044605e-16 -1.94289029e-16 -2.77555756e-17  3.60822483e-16
   6.20385459e-25]
 [-2.22044605e-16  0.00000000e+00 -5.55111512e-17  1.11022302e-16
  -4.13590306e-25]
 [ 0.00000000e+00 -8.32667268e-17  1.19355674e+00  7.49296203e-09
   4.13590306e-25]
 [ 3.88578059e-16  8.32667268e-17  7.49296197e-09 -2.22044605e-16
   4.13590306e-25]
 [ 8.27180613e-25 -4.13590306e-25 -5.67859491e-22  4.13590306e-25
   0.00000000e+00]]
step -----------
P:
[[ 1.12349293e+00 -1.23492927e-01  1.91960814e-01 -1.91960812e-01
  -1.59607038e-09]
 [-1.23492927e-

In [ ]:
# error computations

In [ ]:
# compute squared error for the overlapping time period -------------------------

left = predicted_seiir_prior.index[0]
right = d

seiir_err = case_ma7_all.loc[left:right] - predicted_seiir_prior[left:right]
seiir_sum_sq_err = (seiir_err**2).sum()
seiir_mse = seiir_sum_sq_err / seiir_err.count()

kalman_err = predicted_case.loc[left:right] - case_ma7_all.loc[left:right]
kalman_sum_sq_err = (kalman_err**2).sum()
kalman_mse = kalman_sum_sq_err / kalman_err.count()


print('SSE between seiir forecast and case rate:', seiir_sum_sq_err)
print('MSE between seiir forecast and measured case rate:', seiir_mse)
print()
print('SSE between kalman forecast and case rate:', kalman_sum_sq_err)
print('MSE between kalman forecast and measured case rate:', kalman_mse)

In [ ]:
# MASE calculation -----------------

left = predicted_case.index[0]
right = predicted_case.index[-1]

# take error between prediction and actual
e = (case_ma7_all.loc[left:right] - predicted_case.loc[left:right]).abs()

case_ma7_diff = case_ma7_all.diff(7)
denom = (case_ma7_diff.loc[left:right]).abs()

term = e.div(denom)
term.replace([np.inf, -np.inf], np.nan, inplace=True)
term.dropna(inplace=True)
mase = term.sum() / len(term)
print('mase:', mase)

# All plotting code below this

In [ ]:
# Plotting constants and variables ----------------
matplotlib.rcParams.update({'font.size': 22})
plt.style.use('seaborn-whitegrid')
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]
special_start = datetime.date(2020, 8, 16)

week_interval = pd.date_range(start=special_start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]

In [ ]:
# Single plot with all lines


fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[special_start:d].index, case_ma7_all.loc[special_start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.loc[special_start:tick_end].index, predicted_seiir_prior.loc[special_start:tick_end],
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.loc[special_start:tick_end].index, predicted_case.loc[special_start:tick_end], label='Our 7-Day Forecast',
         c=purple, linewidth=width)
plt.plot(case_ma7_all.loc[d:tick_end].index, case_ma7_all.loc[d:tick_end], c='darkorange',
         linewidth=width)
plt.ylim(-50, 1300)
#plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.xlim(special_start, datetime.date(2020, 11, 3))
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(num_stl_ma7.loc[special_start:d].index, num_stl_ma7.loc[special_start:d], c='red', label='Facebook Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.plot(num_stl_ma7.loc[d:tick_end].index, num_stl_ma7.loc[d:tick_end], c='maroon',
         linewidth=width)
#plt.plot(prop_ma7.index, num_stl_ma7, c='red', label='Facebook Positive Symptoms',
#         linewidth=width)
#plt.ylim(-.065, 2.5)
plt.ylim(-.4, 11)
plt.grid(axis='y', linestyle=':')
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')

plt.show()

In [ ]:
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -700
leftylim_high = 20000

rightylim_low = -3
rightylim_high = 85

xtick_size = 14
xlabel_size = 14

In [8]:
# plot findings -- multiple plots

# Plotting constants and variables ----------------
matplotlib.rcParams.update({'font.size': 18})
plt.style.use('seaborn-whitegrid')
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]


fig1, ax11 = plt.subplots(1)
plt.sca(ax11)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


fig2, ax21 = plt.subplots(1)
plt.sca(ax21)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)




fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(num_stl_ma7.loc[start:d].index, num_stl_ma7.loc[start:d], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax31)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)



fig4, ax41 = plt.subplots(1)
plt.sca(ax41)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax42 = ax41.twinx()
plt.sca(ax42)
plt.plot(num_stl_ma7.loc[start:d].index, num_stl_ma7.loc[start:d], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax41)
plt.plot(predicted_case.loc[start:].index, predicted_case.loc[start:], label='Our 7-Day Forecast',
         c=purple, linewidth=width)

plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


(737517.0, 737732.0)

In [ ]:
fig5, ax51 = plt.subplots(1)
plt.sca(ax51)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)

plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax52 = ax51.twinx()
plt.sca(ax52)
plt.plot(num_stl_ma7.loc[start:d].index, num_stl_ma7.loc[start:d], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax51)
plt.plot(predicted_case.loc[start:].index, predicted_case.loc[start:], label='Our 7-Day Forecast',
         c=purple, linewidth=width)
plt.plot(case_ma7_all.loc[d:tick_end].index, case_ma7_all.loc[d:tick_end], label='Confirmed Case Count', c='yellow',
         linewidth=width)

plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


In [ ]:
plt.show()

In [9]:
## special end date

# plot findings -- multiple plots

# Plotting constants and variables ----------------
matplotlib.rcParams.update({'font.size': 18})
plt.style.use('seaborn-whitegrid')
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]


fig1, ax11 = plt.subplots(1)
plt.sca(ax11)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


fig2, ax21 = plt.subplots(1)
plt.sca(ax21)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)




fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(num_stl_ma7.loc[start:tick_end].index, num_stl_ma7.loc[start:tick_end], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax31)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)



fig4, ax41 = plt.subplots(1)
plt.sca(ax41)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax42 = ax41.twinx()
plt.sca(ax42)
plt.plot(num_stl_ma7.loc[start:tick_end].index, num_stl_ma7.loc[start:tick_end], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax41)
plt.plot(predicted_case.loc[start:].index, predicted_case.loc[start:], label='Our 7-Day Forecast',
         c=purple, linewidth=width)

plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


(737517.0, 737732.0)

In [ ]:
# SHORTENED Erie plot settings
xlim_left = datetime.date(2020, 5, 24)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -72
leftylim_high = 900

rightylim_low = -1
rightylim_high = 12.5

xtick_size = 14
xlabel_size = 14

In [ ]:
# SHORTENED Albany plot settings
xlim_left = datetime.date(2020, 5, 24)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -20
leftylim_high = 275

rightylim_low = -.225
rightylim_high = 3.1

xtick_size = 14
xlabel_size = 14

In [ ]:
# SHORTENED Westchester plot settings
xlim_left = datetime.date(2020, 5, 24)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -100
leftylim_high = 900

rightylim_low = -.75
rightylim_high = 7

xtick_size = 14
xlabel_size = 14

In [ ]:
# SHORTENED Nassau plot settings
xlim_left = datetime.date(2020, 5, 24)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -110
leftylim_high = 1200

rightylim_low = -1.1
rightylim_high = 12

xtick_size = 14
xlabel_size = 14

In [ ]:
# SHORTENED NYC plot settings

xlim_left = datetime.date(2020, 5, 24)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -295
leftylim_high = 8000

rightylim_low = -3
rightylim_high = 80

xtick_size = 14
xlabel_size = 14

In [ ]:
# NYC plot settings

xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -700
leftylim_high = 12000

rightylim_low = -5
rightylim_high = 85

xtick_size = 14
xlabel_size = 14

In [6]:
# Nassau plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -40
leftylim_high = 1650

rightylim_low = -.3
rightylim_high = 12

xtick_size = 14
xlabel_size = 14

In [ ]:
# Westchester plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -28
leftylim_high = 1000

rightylim_low = -.2
rightylim_high = 7

xtick_size = 14
xlabel_size = 14

In [ ]:
# Albany plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -3
leftylim_high = 75

rightylim_low = -.125
rightylim_high = 3.1

xtick_size = 14
xlabel_size = 14

In [ ]:
# Erie plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -10
leftylim_high = 250

rightylim_low = -.5
rightylim_high = 12.5

xtick_size = 14
xlabel_size = 14